Objetivo: ¿Cuáles son los énfasis temáticos y las similitudes o diferencias principales entre los programas de gobierno de estos partidos políticos en Uruguay?

In [ ]:
pip install pdfplumber # esta es una libreria que precisamos para convertir pdfs a texto que podamos leer.

In [ ]:
import requests
import pdfplumber
from io import BytesIO

In [ ]:
# URLs originales
urls = [
    'https://www.frenteamplio.uy/wp-content/uploads/Bases-Programaticas-2025-2030.pdf',
    'https://alvarodelgado.uy/pdf/programa-gobierno.pdf',
    'https://programapartidocolorado.uy/wp-content/uploads/2024/08/LIBRO-PROGRAMA-UNICO-DE-GOBIERNO-2025-PARTIDO-COLORADO.pdf',
    'https://partidoindependiente.uy/wp-content/uploads/2024/05/Programa.-pdf-1.pdf',
    'https://cabildoabierto.uy/wp-content/uploads/2024/08/Programa-CA-2025-2030.pdf'
]

# Nombres descriptivos
nombres_partidos = [
    'Frente Amplio',
    'Partido Nacional',
    'Partido Colorado',
    'Partido Independiente',
    'Cabildo Abierto'
]

# Diccionario para guardar textos en memoria
textos_programas = {}

# Descargar y extraer texto
for url, nombre in zip(urls, nombres_partidos):
    try:
        response = requests.get(url)
        pdf_bytes = BytesIO(response.content)

        texto_completo = ""
        with pdfplumber.open(pdf_bytes) as pdf:
            for page in pdf.pages:
                texto_extraido = page.extract_text()
                if texto_extraido:
                    texto_completo += texto_extraido + "\n"

        textos_programas[nombre] = texto_completo
        print(f"Texto extraído correctamente: {nombre}")

    except Exception as e:
        print(f"Error con {nombre}: {e}")

In [ ]:
print(textos_programas['Cabildo Abierto'])

In [ ]:
for i in nombres_partidos:
    print(i, len(textos_programas[i]))

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    tokens = texto.lower().replace(',', '').replace('¿', '').replace('.', '').replace('!', '').replace('?', '').split()
    tokens = [palabra for palabra in tokens if palabra.isalpha() and palabra not in stop_words]
    return tokens

tokens_programas = {nombre: limpiar_texto(texto) for nombre, texto in textos_programas.items()}

# Ejemplo visual
for partido, tokens in tokens_programas.items():
    print(f"\n{partido} tokens iniciales:", tokens[:10])


In [ ]:
print(textos_programas['Frente Amplio'][0:500])

In [ ]:
print(tokens_programas['Frente Amplio'][0:500])

In [ ]:
from nltk.stem import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer('spanish')

# Aplicar stemming
tokens_stem = {partido: [stemmer.stem(palabra) for palabra in tokens]
               for partido, tokens in tokens_programas.items()}

# Mostrar ejemplo
for partido, stems in tokens_stem.items():
    print(f"{partido} stemming:", stems[:10])


In [ ]:
print(tokens_stem['Frente Amplio'][0:500])

In [ ]:
from collections import Counter
import pandas as pd


In [ ]:
# DTM con stems
dtm = pd.DataFrame({partido: Counter(tokens) for partido, tokens in tokens_stem.items()})
dtm = dtm.fillna(0).astype(int).T

# Inspeccionar primeras columnas
print(dtm.iloc[:, :10])


In [ ]:
# Palabras más frecuentes por partido desde la DTM
for partido in dtm.index:
    print(f"\n🔹 Las 10 palabras más frecuentes en {partido}:")
    print(dtm.loc[partido].sort_values(ascending=False).head(10))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Preparar textos para TF-IDF
textos_stem_final = [" ".join(tokens) for tokens in tokens_stem.values()]

vectorizer = TfidfVectorizer(max_features=30)
X = vectorizer.fit_transform(textos_stem_final)

df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=nombres_partidos)

# Mostrar resultados
print(df_tfidf.round(2))


In [ ]:
# Palabras más distintivas (TF-IDF más alto) por partido
for partido in df_tfidf.index:
    print(f"\n Las 10 palabras más distintivas en {partido} según TF-IDF:")
    print(df_tfidf.loc[partido].sort_values(ascending=False).head(10))


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Matriz de similitud
sim_matrix = cosine_similarity(df_tfidf)
sim_df = pd.DataFrame(sim_matrix, index=nombres_partidos, columns=nombres_partidos)

print("\nSimilitud entre programas:")
print(sim_df.round(2))


In [ ]:
import numpy as np

In [ ]:
# Hacer una copia y reemplazar la diagonal (similitud consigo mismo) por ceros
sim_df_sin_diag = sim_df.copy()
np.fill_diagonal(sim_df_sin_diag.values, 0)

# Encontrar los índices del valor máximo (más parecido)
max_similaridad = sim_df_sin_diag.values.max()
partidos_similares = np.where(sim_df_sin_diag == max_similaridad)

# Obtener los nombres de los partidos
partido_1 = sim_df.index[partidos_similares[0][0]]
partido_2 = sim_df.columns[partidos_similares[1][0]]

print(f"Los dos partidos más similares son '{partido_1}' y '{partido_2}' con una similitud de {max_similaridad:.2f}.")


In [ ]:
# Tarea
## Descargar el catalogo de canciones de The Beatles. https://raw.githubusercontent.com/skut21x-ga/beatles-api-repo/refs/heads/master/db/lyrics.json
## Quedarse solo con las canciones que son de The Beatles (no incluir las de cada Beatle por separado)
## Pre-procesarlas (tener cuidado que al remover stopwords, descargar aquellas en English). No es necesario Stem.
## Encontrar las dos canciones que sean las más parecidas entre ellas.